In [1]:
from __init__ import *
from dataset import IdiomDataset
from collate import collate
from model import IdiomExtractor
from bert_embedder import BERTEmbedder
from hparams import HParams
from trainer import Trainer
from utils import *

SEED = 2
# set seeds to get reproducible results
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
# gpuda bazen randomluk olabiliyormuş onu kaldırmak için
torch.backends.cudnn.deterministic = True

# create bert
it_model_name = 'bert-base-multilingual-cased'
# output hidden states -> it helps to get hidden states from bert
it_config = BertConfig.from_pretrained(it_model_name, output_hidden_states=True)
it_tokenizer = BertTokenizer.from_pretrained(it_model_name)
# get bert weights
hf_it_model = BertModel.from_pretrained(it_model_name, config=it_config)


# Türkçe BERT
tr_model_name = "dbmdz/bert-base-turkish-128k-cased"
tr_config = BertConfig.from_pretrained(tr_model_name, output_hidden_states=True)
tr_tokenizer = BertTokenizer.from_pretrained(tr_model_name)
hf_tr_model = BertModel.from_pretrained(tr_model_name, config=tr_config)

# train, update or test mode selection
#mode = input("Do you want to train or test the model? (train, update, test): ").strip().lower()
mode = "train"
assert mode in ['train', 'update', 'test'], "Mode must be one of train, update, test"
# select the dataset
#dataset_selection = input("Select the dataset (ID10M, ITU, PARSEME, ALL_COMBINED): ").strip().upper()
dataset_selection = "COMBINED"
assert dataset_selection in ['ID10M', 'ITU', 'PARSEME', 'COMBINED'], "Dataset must be one of ID10M, ITU, PARSEME, COMBINED"

# check dataset path
tr_path = r"./src/checkpoints/tr/"
it_path = r"./src/checkpoints/it/"
os.makedirs(tr_path, exist_ok=True)
os.makedirs(it_path, exist_ok=True)

if mode in ["test","update"]:
    # list available checkpoints
    print("Available tr checkpoints:")
    checkpoints = os.listdir(tr_path)
    for i, checkpoint in enumerate(checkpoints):
        print(f"{i+1}. {checkpoint}")
    print("none")
    # load the model
    checkpoint = input("Enter the checkpoint (without .pt): ").strip()
    if checkpoint == "none":
        tr_path = None
    else:
        tr_path = tr_path + checkpoint + ".pt"
        assert os.path.exists(tr_path), "Model path does not exist"

    print("\n")

    print("Available it checkpoints:")
    checkpoints = os.listdir(it_path)
    for i, checkpoint in enumerate(checkpoints):
        print(f"{i+1}. {checkpoint}")
    print("none")
    # load the model
    checkpoint = input("Enter the checkpoint (without .pt): ").strip()
    if checkpoint == "none":
        it_path = None
    else:
        it_path = it_path + checkpoint + ".pt"
        assert os.path.exists(it_path), "Model path does not exist"

model_name = None
if mode in ["train", "update"]:
    #model_name = input("Enter the model name (without .pt): ").strip()
    model_name = "deneme_combined_lstm"

elif mode == "test":
    model_name = checkpoint

# get stanza tagger for both languages
tagger_dict = initialize(use_gpu=True)

# get the path for the dataset
main_path = r"../resources/"+dataset_selection+"/"
train_file = main_path + "train.tsv"
dev_file = main_path + "dev.tsv"
test_file = main_path + "test.tsv"

labels_vocab = {"<pad>":0, "B-IDIOM":1, "I-IDIOM":2, "O":3}

# initialize the dataset
train_dataset, dev_dataset, test_dataset = None, None, None
if mode in ["train", "update"]:
    train_dataset = IdiomDataset(train_file, labels_vocab, tagger_dict)
    dev_dataset = IdiomDataset(dev_file, labels_vocab, tagger_dict)
    print(f"train sentences: {len(train_dataset)}")
    print(f"dev sentences: {len(dev_dataset)}")
    print("-" * 50 + "\n")
else:
    test_dataset = IdiomDataset(test_file, labels_vocab, tagger_dict) 
    print(f"test sentences: {len(test_dataset)}")
    print("-" * 50 + "\n")

#dataloader

if mode in ["train", "update"]:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate)
    dev_dataloader = DataLoader(dev_dataset, batch_size=16, collate_fn=collate)
    print(f"length of train dataloader: {len(train_dataloader)}")
    print(f"length of dev dataloader: {len(dev_dataloader)}")
else:
    test_dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=collate)
    print(f"length of test dataloader: {len(test_dataloader)}")


#instantiate the hyperparameters
params = HParams()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#instantiate the model
it_model = IdiomExtractor(hf_it_model,
                    params).cuda()

it_model.freeze_bert()

tr_model = IdiomExtractor(hf_tr_model,
                    params).cuda()

tr_model.freeze_bert()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

it_embedder =  BERTEmbedder(hf_it_model, it_tokenizer, device)
tr_embedder =  BERTEmbedder(hf_tr_model, tr_tokenizer, device)


2025-05-10 21:06:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Initializing Stanza pipelines...


2025-05-10 21:06:39 INFO: Downloaded file to C:\Users\krbyk\stanza_resources\resources.json
2025-05-10 21:06:39 INFO: Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| lemma     | imst_nocharlm |

2025-05-10 21:06:39 INFO: Using device: cuda
2025-05-10 21:06:39 INFO: Loading: tokenize
2025-05-10 21:06:39 INFO: Loading: mwt
2025-05-10 21:06:40 INFO: Loading: lemma
2025-05-10 21:06:40 INFO: Done loading processors!
2025-05-10 21:06:40 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-05-10 21:06:41 INFO: Downloaded file to C:\Users\krbyk\stanza_resources\resources.json
2025-05-10 21:06:41 INFO: Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nocharlm |

2025-05-10 21:06:41 INFO: Using device: cuda
2025-05-10 21:06:41 INFO: Loading: tokenize
2025-05-10 21:06:41 INFO: Loading: mwt
2025-05-10 21:06:41 INFO: Loading: lemma
2025-05-10 21:06:42 INFO: Done loading processors!


Stanza pipelines initialized.
--------------------------------------------------

Reading dataset...



...

Data encoded.

--------------------------------------------------

Reading dataset...



15311it [00:00, 944813.72it/s]


Dataset read.

--------------------------------------------------

Encoding data...



100%|██████████| 1437/1437 [00:00<00:00, 34965.51it/s]


Data encoded.

--------------------------------------------------

train sentences: 68153
dev sentences: 1437
--------------------------------------------------

length of train dataloader: 4260
length of dev dataloader: 90
BERT model parameters have been frozen.
BERT model parameters have been frozen.


In [2]:
example_sentence = next(iter(train_dataloader))

In [3]:
words, labels, langs = example_sentence

# embedding'i aynı size a getirmek için lazım
# labellar beraber padleniyor embeddingler ayrı
# yani en uzun tr cümle 14, it cümle 12 uzunluğundaysa
# it label 14, it embedding 12 boyutunda oluyor hata veriyor.
global_max = labels.size(1)

print(f"shape of words: {len(words)}, example: {words[0]}")
print(f"shape of labels: {len(labels)}, example: {labels[0]}")
print(f"shape of langs: {len(langs)}, example: {langs}")
print(global_max)

shape of words: 16, example: ['Le', 'differenti', 'strategie', 'riguardo', 'i', 'rifornimenti', 'portano', 'a', 'vari', 'cambi', 'di', 'posizione', 'da', 'cui', 'emergono', 'Räikkönen', 'e', 'Alonso', 'dietro', 'a', 'Fisichella', 'e', 'davanti', 'a', 'Mark', 'Webber', 'su', 'Williams', 'e', 'Jenson', 'Button', 'su', 'BAR', '.']
shape of labels: 16, example: tensor([3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], device='cuda:0')
shape of langs: 16, example: tensor([1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1], device='cuda:0')
75


In [4]:
tr_indices = (langs == 0).nonzero(as_tuple=True)[0]
it_indices = (langs == 1).nonzero(as_tuple=True)[0]
print(f"tr_indices: {tr_indices}")
print(f"it_indices: {it_indices}")

tr_indices: tensor([ 1,  6,  9, 10, 13], device='cuda:0')
it_indices: tensor([ 0,  2,  3,  4,  5,  7,  8, 11, 12, 14, 15], device='cuda:0')


In [5]:
print(f"labels: {labels}")

labels: tensor([[3, 3, 3,  ..., 0, 0, 0],
        [3, 3, 3,  ..., 0, 0, 0],
        [3, 3, 3,  ..., 0, 0, 0],
        ...,
        [3, 3, 3,  ..., 0, 0, 0],
        [3, 1, 2,  ..., 0, 0, 0],
        [3, 3, 3,  ..., 0, 0, 0]], device='cuda:0')


In [6]:
tr_words = [words[i] for i in tr_indices.cpu().numpy()]
tr_labels = labels[tr_indices] # 0 1 5 7 10 11 12. labellar tr dilinin labelları

print(f"length of tr_words: {len(tr_words)}, example: {tr_words[0]}")
print(f"shape of tr_labels: {tr_labels.shape}, example: {tr_labels}")

length of tr_words: 5, example: ['Büro', 'amirim', 'de', 'göndermiş', '.']
shape of tr_labels: torch.Size([5, 75]), example: tensor([[3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        

In [7]:
# embed tr list of embeddings herbirisi 10,seq_length,768 -> [tr_batch_size, seq_length, 768]
tr_embedded = tr_embedder.embed_sentences(tr_words)
print(f"shape of embedded_tr: {len(tr_embedded)}, example: {tr_embedded[0].shape}")
print(f"shape of embedded_tr: {len(tr_embedded)}, example: {tr_embedded[1].shape}")

shape of embedded_tr: 5, example: torch.Size([4, 768])
shape of embedded_tr: 5, example: torch.Size([14, 768])


In [8]:
# tr embedded'i -1le padleyerek seq_lengthi eşitliyoruz. -> [tr_batch_size, max_seq_length, 768]
tr_embs = pad_sequence(tr_embedded, batch_first=True, padding_value=0).to(device)

# check embedding size to match the labels
if tr_embs.size(1) < global_max:
    # ekstra pad lazım
    pad_size = global_max - tr_embs.size(1)

    tr_embs = F.pad(tr_embs, (0, 0, 0, pad_size), "constant", 0)


print(f"shape of tr_embs: {tr_embs.shape}, example: {tr_embs[0]}")

shape of tr_embs: torch.Size([5, 75, 768]), example: tensor([[ 0.0342,  0.1330, -1.0200,  ...,  2.9643,  2.1837, -2.9831],
        [ 1.3881, -0.9751, -1.8090,  ...,  4.8396,  3.2179,  0.6546],
        [ 2.7253, -1.0878, -1.7679,  ...,  3.0010,  3.9376,  2.5786],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')


In [9]:
it_words = [words[i] for i in it_indices.cpu().numpy()]
it_labels = labels[it_indices]

print(f"shape of it_labels: {it_labels.shape}, example: {it_labels}")
print(f"shape of it_words: {len(it_words)}, example: {it_words[0]}")

shape of it_labels: torch.Size([11, 75]), example: tensor([[3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [3, 3, 3, 3, 3, 3, 1, 2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 

In [10]:
# embed tr list of embeddings herbirisi 10,seq_length,768 -> [tr_batch_size, seq_length, 768]
it_embedded = it_embedder.embed_sentences(it_words)
print(f"shape of embedded_it: {len(it_embedded)}, example: {it_embedded[0].shape}")
print(f"shape of embedded_it: {len(it_embedded)}, example: {it_embedded[1].shape}")

shape of embedded_it: 11, example: torch.Size([33, 768])
shape of embedded_it: 11, example: torch.Size([14, 768])


In [11]:
it_embs = pad_sequence(it_embedded, batch_first=True, padding_value=0).to(device)

# check embedding size to match the labels
if it_embs.size(1) < global_max:
    # ekstra pad lazım
    pad_size = global_max - it_embs.size(1)

    it_embs = F.pad(it_embs, (0, 0, 0, pad_size), "constant", 0)

print(f"shape of it_embs: {it_embs.shape}, example: {it_embs[0]}")

shape of it_embs: torch.Size([11, 75, 768]), example: tensor([[-2.9175, -2.8953,  2.5226,  ...,  0.1959,  2.0889,  1.5841],
        [-1.7234, -0.0500, -0.0436,  ...,  1.4306,  1.7430,  0.4612],
        [-6.1008,  1.6713,  4.7920,  ..., -2.6665,  0.9332,  0.7428],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')


In [12]:
tr_LL, _ = tr_model(tr_embs, tr_labels)
tr_NLL = -tr_LL

it_LL,_ = it_model(it_embs, it_labels)
it_NLL = -it_LL

print(f"tr_NLL: {tr_NLL}")
print(f"it_NLL: {it_NLL}")
loss = tr_NLL + it_NLL
print(f"loss: {loss}")

tr_optimizer = optim.Adam(tr_model.parameters(), lr=0.0001)
it_optimizer = optim.Adam(it_model.parameters(), lr=0.0001)

# Optimizer step
tr_optimizer.zero_grad()
it_optimizer.zero_grad()
loss.backward()
torch.nn.utils.clip_grad_norm_(tr_model.parameters(), 1)
torch.nn.utils.clip_grad_norm_(it_model.parameters(), 1)
tr_optimizer.step()
it_optimizer.step()


tr_NLL: 95.6654052734375
it_NLL: 563.3499145507812
loss: 659.0153198242188


In [13]:
# get one batch from dev set
eval_data = next(iter(dev_dataloader))
words, labels, langs = eval_data

# tr ve it nin indexlerini ayıkla
tr_indices = (langs == 0).nonzero(as_tuple=True)[0]
it_indices = (langs == 1).nonzero(as_tuple=True)[0]

batch_size, seq_len = labels.shape
device = labels.device
global_max = seq_len

hidden_size = tr_embedder.bert_model.config.hidden_size

# TR cümleleri ayıkla, embeddle ve petle
tr_words  = [words[i] for i in tr_indices.cpu().numpy()]
tr_labels = labels[tr_indices]
# türkçe cümle gelmiş mi kontrolü
if len(tr_words) > 0:
    tr_embedded = tr_embedder.embed_sentences(tr_words)
    tr_embs     = pad_sequence(tr_embedded, batch_first=True, padding_value=0).to(device)
    if tr_embs.size(1) < global_max:
        # ekstra pad lazım
        pad_size = global_max - tr_embs.size(1)
        tr_embs  = F.pad(tr_embs, (0, 0, 0, pad_size), "constant", 0)
else:
    # hiç TR cümlesi yoksa boş batch oluştur
    tr_embs = torch.zeros((0, global_max, hidden_size), device=device)

# IT cümleleri ayıkla, embeddle ve petle
it_words    = [words[i] for i in it_indices.cpu().numpy()]
it_labels   = labels[it_indices]
if len(it_words) > 0:
    it_embedded = it_embedder.embed_sentences(it_words)
    it_embs     = pad_sequence(it_embedded, batch_first=True, padding_value=0).to(device)
    if it_embs.size(1) < global_max:
        # ekstra pad lazım
        pad_size = global_max - it_embs.size(1)
        it_embs  = F.pad(it_embs, (0, 0, 0, pad_size), "constant", 0)
else:
    # hiç IT cümlesi yoksa boş batch oluştur
    it_embs = torch.zeros((0, global_max, hidden_size), device=device)

# take predictions
tr_decode = tr_model(tr_embs, None)
print(f"tr_decode: {tr_decode}")
it_decode = it_model(it_embs, None)

# alt kısım GPT bakmak lazım 

def decode_to_tensor(decode_out, seq_len, device):
    # 1) list of lists → list of 1D tensors
    token_tensors = [torch.tensor(seq, dtype=torch.long, device=device)
                     for seq in decode_out]
    # 2) hiç prediction yoksa boş tensor
    if not token_tensors:
        return torch.zeros((0, seq_len), dtype=torch.long, device=device)
    # 3) pad_sequence ile batch_first ve padding_value=-1
    padded = pad_sequence(token_tensors, batch_first=True, padding_value=-1)
    # 4) eğer hâlâ seq_len’den kısa ise sağa pad et
    if padded.size(1) < seq_len:
        pad_amt = seq_len - padded.size(1)
        padded = F.pad(padded, (0, pad_amt), value=-1)
    return padded

# kullanım:
tr_pred = decode_to_tensor(tr_decode, seq_len, device)
it_pred = decode_to_tensor(it_decode, seq_len, device)


# list of lists → tensor’a çevir
if len(tr_decode) > 0:
    tr_pred = torch.full((len(tr_decode), seq_len),
                         fill_value=-1, dtype=torch.long, device=device)
    for i, seq in enumerate(tr_decode):
        tr_pred[i, :len(seq)] = torch.tensor(seq, device=device)
else:
    tr_pred = torch.zeros((0, seq_len), dtype=torch.long, device=device)

if len(it_decode) > 0:
    it_pred = torch.full((len(it_decode), seq_len),
                         fill_value=-1, dtype=torch.long, device=device)
    for i, seq in enumerate(it_decode):
        it_pred[i, :len(seq)] = torch.tensor(seq, device=device)
else:
    it_pred = torch.zeros((0, seq_len), dtype=torch.long, device=device)

# 1) Tam batch için pred tensor’ü oluştur
all_pred = torch.full(
    (batch_size, seq_len),
    fill_value=-1,                # pad bölgelerde -1 kalsın
    dtype=torch.long,
    device=device
)

# 2) TR ve IT tahminlerini ilgili index’lere yerleştir
all_pred[tr_indices] = tr_pred
all_pred[it_indices] = it_pred

# 3) Sadece gerçek token’lar üzerinde doğru/yanlış kontrolü
valid_mask   = labels.ne(-1)        # True olan yerler gerçek token
correct_mask = (all_pred == labels) & valid_mask

# 4) İsterseniz token‐seviye accuracy
token_accuracy = correct_mask.sum().float() / valid_mask.sum().float()
print(f"Token‐seviye accuracy: {token_accuracy:.4f}")

# 5) Veya örnek‐seviye (sentence‐seviye) doğru karar:
#    her cümlenin tüm token’ları doğru mu?
sent_correct     = correct_mask.all(dim=1) & valid_mask.any(dim=1)
# valid_mask.any(dim=1) ile “tamamen pad’li” örnekleri atlıyoruz
sentence_accuracy = sent_correct.sum().float() / sent_correct.numel()
print(f"Sentence‐seviye accuracy: {sentence_accuracy:.4f}")

# all_pred şimdi (batch_size, seq_len) shape’inde, 
# her satır orijinal sırasıyla tahminleri içeriyor.


tr_decode: [[3, 2, 3, 0, 3, 2, 0, 0, 3, 2, 3, 2, 3, 2, 3, 1, 2, 2], [2, 2, 3, 2, 3, 2, 0, 3, 0, 3, 0, 0, 0, 3, 2, 3, 0, 2], [1, 3, 1, 1, 2, 2, 3, 2, 1, 3, 0, 0, 3, 0, 0, 0, 3, 0], [2, 0, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 0, 3, 2, 3, 1], [3, 0, 3, 3, 0, 3, 0, 3, 0, 3, 3, 0, 1, 2, 2, 2, 3, 3], [0, 3, 2, 0, 1, 3, 1, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3], [2, 2, 0, 1, 3, 2, 3, 0, 1, 0, 3, 0, 3, 3, 2, 3, 2, 0], [2, 0, 0, 0, 0, 2, 2, 0, 3, 2, 0, 2, 3, 3, 3, 0, 0, 0], [3, 0, 0, 2, 2, 0, 0, 0, 3, 0, 2, 0, 0, 3, 2, 3, 0, 3], [0, 0, 1, 0, 3, 1, 1, 3, 2, 1, 3, 0, 2, 3, 2, 0, 3, 0], [1, 2, 2, 3, 0, 3, 2, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0], [3, 0, 3, 0, 2, 2, 2, 3, 1, 0, 0, 0, 3, 0, 0, 3, 0, 3], [0, 2, 3, 2, 0, 0, 0, 1, 1, 3, 3, 0, 3, 0, 1, 2, 3, 3], [1, 3, 1, 1, 0, 2, 3, 1, 0, 0, 0, 0, 3, 2, 0, 1, 2, 2], [2, 0, 0, 3, 0, 0, 0, 1, 1, 3, 0, 1, 0, 3, 3, 0, 0, 2], [1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 3, 0, 3]]
Token‐seviye accuracy: 0.3299
Sentence‐seviye accuracy: 0.0000


In [14]:
trainer = Trainer(tr_model = tr_model, it_model = it_model,
                tr_optimizer = optim.Adam(tr_model.parameters(), lr=0.0001),
                it_optimizer = optim.Adam(it_model.parameters(), lr=0.0001),
                tr_embedder= tr_embedder,
                it_embedder= it_embedder,
                modelname = model_name,
                labels_vocab=labels_vocab)

trainer.train(train_dataloader, dev_dataloader, 50, patience=10)



Training...
 Epoch 001, patience: 10


Epoch 1:  75%|███████▍  | 3194/4260 [24:00<08:00,  2.22it/s]  


KeyboardInterrupt: 